# 1. Import Libraries

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import requests
from io import BytesIO
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, EfficientNetB3, DenseNet169, Xception, ConvNeXtBase
from tensorflow.keras.models import Model
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

SEED = 244
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 2. Load Dataset

### 2.1 Define GitHub Repo & Folder Path

In [2]:
GITHUB_REPO = "prattapong/Commercial-Airplane-Model-Image-Classification"
GITHUB_FOLDER = "images"
GITHUB_API_URL = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{GITHUB_FOLDER}"

### 2.2 Fetch Image URLs and Load Image

In [3]:
# Fetch Image URLs Automatically
def get_image_urls():
    response = requests.get(GITHUB_API_URL)
    if response.status_code != 200:
        raise Exception(f"Error fetching images: {response.json()}")

    image_urls = {}
    for folder in response.json():
        if folder["type"] == "dir":  # Ensure it's a folder (A350, B787, A320)
            class_name = folder["name"]
            image_urls[class_name] = []
            folder_url = folder["url"]

            # Fetch image files in each class folder
            folder_response = requests.get(folder_url)
            if folder_response.status_code == 200:
                for file in folder_response.json():
                    if file["name"].lower().endswith((".jpg", ".jpeg", ".png")):
                        image_urls[class_name].append(file["download_url"])

    return image_urls

# Load Images Using Image.open()
def load_images(image_urls):
    IMG_SIZE = (224, 224)  # Resize all images to 224x224
    X, y = [], []

    total_images = sum(len(urls) for urls in image_urls.values())  # Total number of images
    progress_bar = tqdm(total=total_images, desc="Loading Images", unit="img")

    for label, urls in image_urls.items():
        for url in urls:
            try:
                response = requests.get(url)
                img = Image.open(BytesIO(response.content)).convert("RGB")  # Load image
                img = img.resize(IMG_SIZE)
                X.append(np.array(img) / 255.0)  # Normalize
                y.append(label)
            except Exception as e:
                print(f"Error loading {url}: {e}")
            progress_bar.update(1)  # Update progress bar

    progress_bar.close()
    return np.array(X), pd.Categorical(y).codes

In [ ]:
image_urls = get_image_urls()
X, y = load_images(image_urls)

Loading Images:  10%|▉         | 58/605 [00:16<02:23,  3.82img/s]

# 3. Data-preprocessing

In [ ]:
def augment_data(X_train):
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest"
    )

    datagen.fit(X_train)

    return datagen

# 4. Train model

### 4.1 List 5 pre-trained models

In [ ]:
MODELS = {
    "ResNet50": ResNet50,
    "EfficientNetB3": EfficientNetB3,
    "DenseNet169": DenseNet169,
    "Xception": Xception,
    "ConvNeXtBase": ConvNeXtBase
}

### 4.2 Initiate training loop

In [ ]:
def train_and_evaluate_model(model_name, X, y, batch_size, epochs, learning_rate, dropout_rate, num_unfreezed_layer=0):
    print(f'*** Training {model_name} ***')
    print(f'- Batch Size: {batch_size}')
    print(f'- LR: {learning_rate}')
    print(f'- Dropout: {dropout_rate}')
    print(f'- Last "{num_unfreezed_layer}" layers unfreezed\n')

    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=SEED
    )

    # Define Input Layer
    input_tensor = Input(shape=(224, 224, 3))

    # Load Pretrained Model
    base_model = MODELS[model_name](weights="imagenet", include_top=False, input_tensor=input_tensor)

    # Unfreeze Layers If Needed
    if num_unfreezed_layer == 0:
        for layer in base_model.layers:
            layer.trainable = False
    else:
        for layer in base_model.layers[-num_unfreezed_layer:]:
            layer.trainable = True

    # Add Custom Classification Head
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(128, activation="relu")(x)
    x = Dropout(dropout_rate)(x)
    output_layer = Dense(len(np.unique(y)), activation="softmax")(x)

    # Build Model
    model = Model(inputs=input_tensor, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

    # Data Augmentation
    datagen = augment_data(X_train)

    # Model Checkpointing
    model_path = f'best_{model_name}.h5'  # Use .keras format
    checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

    # Train Model
    history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                        validation_data=(X_test, y_test),
                        epochs=epochs,
                        callbacks=[checkpoint])

    # Load Best Model (Fix for ConvNeXtBase)
    custom_objects = get_custom_objects() if model_name == "ConvNeXtBase" else None
    best_model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)

    # Evaluate on Test Data
    test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
    print(f"\n{model_name} Test Accuracy: {test_accuracy:.4f}")

    return model_name, test_accuracy, history

def compare_models(X, y, learning_rates, dropout_rates, batch_sizes, num_unfreezed_layers, epochs=50):
    results = []

    # Loop through all model hyperparameters with tqdm progress bar
    for model_name in MODELS.keys():
        for lr in learning_rates:
            for dr in dropout_rates:
                for batch_size in batch_sizes:
                    for num_unfreezed_layer in num_unfreezed_layers:
                        result = train_and_evaluate_model(model_name, X, y, batch_size, epochs, lr, dr, num_unfreezed_layer)
                        results.append(result)
                        print('------------------------------------------------------------------------\n')

    # Sort models by accuracy
    results.sort(key=lambda x: x[1], reverse=True)
    print("\nModel Performance Ranking:")
    for rank, (model_name, accuracy, _) in enumerate(results, 1):
        print(f"{rank}. {model_name}: {accuracy:.4f}")

    return results

In [ ]:
# image_urls = get_image_urls()
# X, y = load_images(image_urls)

### 4.3 Set Hyper-parameter

In [ ]:
num_unfreezed_layers = [0]
learning_rates = [0.0001]
dropout_rates = [0.2]
batch_sizes = [8]

### 4.4 Train all models and parameters

In [ ]:
best_model = compare_models(
    X = X,
    y = y,
    epochs = 5,

    ############## Hyper-parameter ##############
    learning_rates = learning_rates,
    dropout_rates = dropout_rates,
    batch_sizes = batch_sizes,
    num_unfreezed_layers = num_unfreezed_layers
)

# 5. Show result of a sample

In [21]:
def show_prediction(model, X, y, index, class_labels):
    """
    Displays an image from X[index] with the model's predicted and actual label.

    Parameters:
    - model: Trained Keras model
    - X: NumPy array of images
    - y: NumPy array of actual labels
    - index: Index of the image to display
    - class_labels: List of class names corresponding to label indices
    """
    sample_image = np.array(X[index])
    sample_input = np.expand_dims(sample_image, axis=0)

    predicted_class = np.argmax(model.predict(sample_input))
    actual_class = y[index]

    predicted_label = class_labels[predicted_class]
    actual_label = class_labels[actual_class]

    # Display the image with prediction
    plt.imshow(sample_image)
    plt.axis("off")
    plt.title(f"Predicted: {predicted_label}\nActual: {actual_label}", fontsize=12, color="blue")
    plt.show()

In [25]:
class_labels = ["Airbus A320", "Airbus A350", "Boeing 787"]
best_model = tf.keras.models.load_model("best_DenseNet169.h5")

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'best_DenseNet169.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [23]:
for i in [0,10,402,405,500, 600]:
    show_prediction(best_model, X, y, index=i, class_labels=class_labels)

AttributeError: 'list' object has no attribute 'predict'

In [24]:
best_model

[('DenseNet169',
  0.6363636255264282,
  <keras.src.callbacks.history.History at 0x78474e749150>),
 ('Xception',
  0.4958677589893341,
  <keras.src.callbacks.history.History at 0x784605569ed0>),
 ('ConvNeXtBase',
  0.4628099203109741,
  <keras.src.callbacks.history.History at 0x78460436d1d0>),
 ('ResNet50',
  0.40495866537094116,
  <keras.src.callbacks.history.History at 0x7847c8233890>),
 ('EfficientNetB3',
  0.35537189245224,
  <keras.src.callbacks.history.History at 0x7847b41251d0>)]